In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
from pathlib import Path
import csv
import pandas as pd
import time

options = Options()
options.add_argument('-headless')
driver = webdriver.Firefox(service_log_path=Path("geckodriver.log"), options=options, service=GeckoDriverManager().install())

df = pd.DataFrame(columns=["Number", "Title", "Year", "Runtime", "Genre", "Rating", "Budget", "Votes", "Metascore"])

url_template = "https://www.imdb.com/search/title/?genres=sci-fi&start={}&explore=title_type,genres&title_type=movie&ref_=adv_explore_rhs&language=en"
start = 1
movies = []

# sadece ilk 2 sayfanın verisini almak için döngüyü 2 kez çalıştırıyoruz
for start in range(1, 6001, 50):
    url = url_template.format(start)
    driver.get(url)
    time.sleep(6)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    items = soup.select('.lister-item-content')
    if not items:
        break


    for item in soup.select('.lister-item-content'):
        try:
            number = item.select_one('.lister-item-header span').text.strip('()')
        except IndexError:
            number = None
        
        try:
            title = item.select_one('.lister-item-header a').text.strip()
        except IndexError:
            title = None
        
        try:
            runtime_element = item.select_one('.runtime')
        except IndexError:
            runtime_element = None
            
        try:
            runtime = runtime_element.text.strip() if runtime_element else ''
        except IndexError:
            runtime = None
        
        try:
            rating_element = item.select_one('.ratings-imdb-rating strong')
        except IndexError:
            rating_element = None
            
        try:
            rating = rating_element.text.strip() if rating_element else ''
        except IndexError:
            rating = None
            
        try:
            year = item.select_one('.lister-item-content .text-muted').text.strip()
        except IndexError:
            year = None
        
        try:
            genre_element = item.select_one('.genre')
        except IndexError:
            genre_element = None
        
        try:
            genre = " - ".join(genre_element.text.strip().split(',')) if genre_element else ''
        except IndexError:
            genre = None

        # Metascore bilgisini alınacaktır.
        try:
            metascore_element = item.select_one('.metascore')
        except IndexError:
            metascore_element = None
        
        try:
            metascore = metascore_element.text.strip() if metascore_element else ''
        except IndexError:
            metascore = None

        try:
            link = item.select_one('.lister-item-header a')['href']
        except IndexError:
            link = None
            
        driver.get(f"https://www.imdb.com{link}business")
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
            
        # Oyların sayısı alınacaktır.
        votes_element = item.select_one('span[name="nv"]')
        if votes_element:
            votes = votes_element['data-value']
        else:
            votes = ''
        
        # Budget değeri varsa alınacak eğer tahmini varsa o da alınacaktır.
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        budget_elements = soup.findAll('span', {'class': 'ipc-metadata-list-item__list-content-item'})
        budget = ''
        for element in budget_elements:
            text = element.text.strip()
            if text.startswith('$'):
                if any(x in text.lower() for x in ["gross worldwide", "gross us & canada", "opening weekend us & canada"]):
                    continue
                else:
                    if "Budget" in text.lower() or "estimated" in text.lower():
                        budget = text.split(' (')[0]
                        break
                    else:
                        budget = ''
            elif "Budget" in text.lower() or "estimated" in text.lower():
                    budget = text.split(' (')[0]
                    break
            else:
                budget = ''

        df = pd.concat([df, pd.DataFrame({"Number": number, "Title": title, "Year": year, "Runtime": runtime, "Genre": genre, "Rating": rating, "Budget": budget, "Votes": votes, "Metascore": metascore}, index=[0])], ignore_index=True)
        df.to_csv("imdb_scraped_data_scifi.csv", index=False)
driver.quit()

In [16]:
df

,Number,Title,Year,Runtime,Genre,Rating,Budget,Votes,Metascore
0,1.,Guardians of the Galaxy Vol. 3,(2023),150 min,Action - Adventure - Comedy,8.4,,60367,65
1,2.,Dune: Part Two,(2023),,Action - Adventure - Drama,,,,
2,3.,The Hunger Games: The Ballad of Songbirds and ...,(2023),,Action - Adventure - Drama,,,,
3,4.,The Flash,(2023),,Action - Adventure - Fantasy,,"$200,000,000",,
4,5.,Guardians of the Galaxy,(2014),121 min,Action - Adventure - Comedy,8.0,"$170,000,000",1206028,76
5,6.,Dune,(2021),155 min,Action - Adventure - Drama,8.0,"$165,000,000",669646,74
6,7.,Avatar: The Way of Water,(2022),192 min,Action - Adventure - Fantasy,7.7,"$350,000,000",373613,67
7,8.,Ant-Man and the Wasp: Quantumania,(2023),124 min,Action - Adventure - Comedy,6.2,"$200,000,000",128209,48
8,9.,Transformers: Rise of the Beasts,(2023),,Action - Adventure - Sci-Fi,,"$200,000,000",,
9,10.,Guardians of the Galaxy Vol. 2,(2017),136 min,Action - Adventure - Comedy,7.6,"$200,000,000",703992,67


In [3]:
df.head(55)

,Number,Title,Year,Runtime,Genre,Rating,Budget,Votes,Metascore
0,1.,The Hunger Games: The Ballad of Songbirds and ...,(2023),,Action - Adventure - Drama,,,,
1,2.,The Flash,(2023),,Action - Adventure - Fantasy,,"$200,000,000",,
2,3.,Guardians of the Galaxy Vol. 3,(2023),150 min,Action - Adventure - Comedy,8.4,,35997,65
3,4.,Ant-Man and the Wasp: Quantumania,(2023),124 min,Action - Adventure - Comedy,6.2,"$200,000,000",126234,48
4,5.,Transformers: Rise of the Beasts,(2023),,Action - Adventure - Sci-Fi,,"$200,000,000",,
5,6.,Avatar: The Way of Water,(2022),192 min,Action - Adventure - Fantasy,7.7,"$350,000,000",372123,67
6,7.,Mighty Morphin Power Rangers: Once & Always,(2023),55 min,Action - Adventure - Family,5.8,,4766,61
7,8.,Everything Everywhere All at Once,(2022),139 min,Action - Adventure - Comedy,7.9,"$14,300,000",438661,81
8,9.,65,(2023),93 min,Action - Adventure - Drama,5.4,"$45,000,000",39416,40
9,10.,Nope,(2022),130 min,Horror - Mystery - Sci-Fi,6.8,"$68,000,000",224352,77


In [4]:
df.info

<bound method DataFrame.info of       Number                                              Title    Year   
0         1.  The Hunger Games: The Ballad of Songbirds and ...  (2023)  \
1         2.                                          The Flash  (2023)   
2         3.                     Guardians of the Galaxy Vol. 3  (2023)   
3         4.                  Ant-Man and the Wasp: Quantumania  (2023)   
4         5.                   Transformers: Rise of the Beasts  (2023)   
...      ...                                                ...     ...   
5721  5,722.                                        Way Station           
5722  5,723.                                       Fallen Cards           
5723  5,724.                                      Bimbo bambino  (2018)   
5724  5,725.                                            Virus X  (2010)   
5725  5,726.                                             Gogola  (1966)   

      Runtime                           Genre Rating        Budget 

In [5]:
df.describe()

,Number,Title,Year,Runtime,Genre,Rating,Budget,Votes,Metascore
count,5726,5726,5726,5726,5726,5726,5726,5726,5726
unique,5726,5522,191,161,309,90,712,3628,91
top,1.,Dr. Jekyll and Mr. Hyde,,,Horror - Sci-Fi,,,,
freq,1,4,296,530,367,554,3490,554,4358


In [6]:
df.describe().T

,count,unique,top,freq
Number,5726,5726,1.,1
Title,5726,5522,Dr. Jekyll and Mr. Hyde,4
Year,5726,191,,296
Runtime,5726,161,,530
Genre,5726,309,Horror - Sci-Fi,367
Rating,5726,90,,554
Budget,5726,712,,3490
Votes,5726,3628,,554
Metascore,5726,91,,4358


In [7]:
df.isnull().sum()

Number       0
Title        0
Year         0
Runtime      0
Genre        0
Rating       0
Budget       0
Votes        0
Metascore    0
dtype: int64

In [9]:
df.isna().sum()

Number       0
Title        0
Year         0
Runtime      0
Genre        0
Rating       0
Budget       0
Votes        0
Metascore    0
dtype: int64

In [12]:
df.isin([""]).sum()

Number          0
Title           0
Year          296
Runtime       530
Genre           0
Rating        554
Budget       3490
Votes         554
Metascore    4358
dtype: int64

In [13]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time

options = Options()
options.headless = True
driver = webdriver.Firefox(firefox_binary="C:/Program Files/LibreWolf/librewolf.exe", options=options, executable_path=GeckoDriverManager().install())

df = pd.DataFrame(columns=["Number", "Title", "Year", "Runtime", "Genre", "Rating", "Budget", "Votes", "Metascore"])

url_template = "https://www.imdb.com/search/title/?genres=fantasy&start={}&explore=title_type,genres&title_type=movie&ref_=adv_explore_rhs&language=en"
start = 1
movies = []

# sadece ilk 2 sayfanın verisini almak için döngüyü 2 kez çalıştırıyoruz
for start in range(1, 1001, 50):
    url = url_template.format(start)
    driver.get(url)
    time.sleep(6)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    items = soup.select('.lister-item-content')
    if not items:
        break
        
    df = pd.concat([df, pd.DataFrame({"Number": number, "Title": title, "Year": year, "Runtime": runtime, "Genre": genre, "Rating": rating, "Budget": budget, "Votes": votes, "Metascore": metascore}, index=[0])], ignore_index=True)


    for item in soup.select('.lister-item-content'):
        number = item.select_one('.lister-item-header span').text.strip('()')
        title = item.select_one('.lister-item-header a').text.strip()
        runtime_element = item.select_one('.runtime')
        runtime = runtime_element.text.strip() if runtime_element else ''
        rating_element = item.select_one('.ratings-imdb-rating strong')
        rating = rating_element.text.strip() if rating_element else ''
        year = item.select_one('.lister-item-content .text-muted').text.strip()

        genre_element = item.select_one('.genre')
        genre = " - ".join(genre_element.text.strip().split(',')) if genre_element else ''

        # Metascore bilgisini alınacaktır.
        metascore_element = item.select_one('.metascore')
        metascore = metascore_element.text.strip() if metascore_element else ''


        link = item.select_one('.lister-item-header a')['href']
        driver.get(f"https://www.imdb.com{link}business")
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
            
        # Oyların sayısı alınacaktır.
        votes_element = item.select_one('span[name="nv"]')
        if votes_element:
            votes = votes_element['data-value']
        else:
            votes = ''
        
        # Budget değeri varsa alınacak eğer tahmini varsa o da alınacaktır.
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        budget_elements = soup.findAll('span', {'class': 'ipc-metadata-list-item__list-content-item'})
        budget = ''
        for element in budget_elements:
            text = element.text.strip()
            if text.startswith('$'):
                if any(x in text.lower() for x in ["gross worldwide", "gross us & canada", "opening weekend us & canada"]):
                    continue
                else:
                    if "Budget" in text.lower() or "estimated" in text.lower():
                        budget = text.split(' (')[0]
                        break
                    else:
                        budget = ''
            elif "Budget" in text.lower() or "estimated" in text.lower():
                    budget = text.split(' (')[0]
                    break
            else:
                budget = ''

        df = pd.concat([df, pd.DataFrame({"Number": number, "Title": title, "Year": year, "Runtime": runtime, "Genre": genre, "Rating": rating, "Budget": budget, "Votes": votes, "Metascore": metascore}, index=[0])], ignore_index=True)
        df.to_csv("imdb_scraped_data_fantasy.csv", index=False)
driver.quit()

C:\Users\Sw4yt\AppData\Local\Temp\ipykernel_13136\1649255778.py:13: DeprecationWarning: headless property is deprecated, instead use add_argument('-headless')
  options.headless = True
C:\Users\Sw4yt\AppData\Local\Temp\ipykernel_13136\1649255778.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(firefox_binary="C:/Program Files/LibreWolf/librewolf.exe", options=options, executable_path=GeckoDriverManager().install())
C:\Users\Sw4yt\AppData\Local\Temp\ipykernel_13136\1649255778.py:14: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(firefox_binary="C:/Program Files/LibreWolf/librewolf.exe", options=options, executable_path=GeckoDriverManager().install())


In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time
from selenium.webdriver.chrome.service import Service


options = Options()
options.add_argument('--headless')
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)


df = pd.DataFrame(columns=["Number", "Title", "Year", "Runtime", "Genre", "Rating", "Budget", "Votes", "Metascore, Gross"])

url_template = "https://www.imdb.com/search/title/?genres=comedy&sort=num_votes,desc&start={}&explore=title_type,genres&title_type=movie&ref_=adv_explore_rhs&language=en"
start = 1
movies = []

# Only scrape data from the first 2 pages, run the loop twice
for start in range(1, 1001, 50):
    url = url_template.format(start)
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    items = soup.select('.lister-item-content')
    if not items:
        break

    for item in soup.select('.lister-item-content'):
        number = item.select_one('.lister-item-header span').text.strip('()')
        title = item.select_one('.lister-item-header a').text.strip()
        runtime_element = item.select_one('.runtime')
        runtime = runtime_element.text.strip() if runtime_element else ''
        rating_element = item.select_one('.ratings-imdb-rating strong')
        rating = rating_element.text.strip() if rating_element else ''
        year = item.select_one('.lister-item-content .text-muted').text.strip()

        genre_element = item.select_one('.genre')
        genre = " - ".join(genre_element.text.strip().split(',')) if genre_element else ''

        # Metascore bilgisini alınacaktır.
        metascore_element = item.select_one('.metascore')
        metascore = metascore_element.text.strip() if metascore_element else ''

        # Oyların sayısı alınacaktır.
        votes_element = item.select_one('span[name="nv"]')
        if votes_element:
            votes = votes_element['data-value']
        else:
            votes = ''

        # Budget değeri varsa alınacak eğer tahmini varsa o da alınacaktır.
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        budget_elements = soup.findAll('span', {'class': 'ipc-metadata-list-item__list-content-item'})
        budget = ''
        for element in budget_elements:
            text = element.text.strip()
            if text.startswith('$'):
                if any(x in text.lower() for x in ["gross worldwide", "gross us & canada", "opening weekend us & canada"]):
                    continue
                else:
                    if "Budget" in text.lower() or "estimated" in text.lower():
                        budget = text.split(' (')[0]
                        break
                    else:
                        budget = ''
            elif "Budget" in text.lower() or "estimated" in text.lower():
                    budget = text.split(' (')[0]
                    break
            else:
                budget = ''

        # Extracting the Gross revenue value
        gross_elements = item.select_one('span[name="nv"]')
        if gross_elements:
            gross_value = gross_elements['data-value']
        else:
            gross_value = ''

        print(f"Title: {title}")
        print(f"Gross: {gross_value}")

df = pd.concat([df, pd.DataFrame({"Number": number, "Title": title, "Year": year, "Runtime": runtime, "Genre": genre, "Rating": rating, "Budget": budget, "Votes": votes, "Metascore": metascore, "Gross": gross_value}, index=[0])], ignore_index=True)


        df = pd.concat([df, pd.DataFrame({"Number": number, "Title": title, "Year": year, "Runtime": runtime, "Genre": genre, "Rating": rating, "Budget": budget, "Votes": votes, "Metascore": metascore, "Gross": gross_value}, index=[0])], ignore_index=True)
        df.to_csv("imdb_scraped_data_comedy.csv", index=False)
    driver.quit()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 94)

In [18]:
import csv
import pandas as pd

df_1 = pd.read_csv("imdb_scraped_data_scifi.csv")
df_2 = pd.read_csv("imdb_scraped_data_comedy.csv")
df_3 = pd.read_csv("imdb_scraped_data_fantasy.csv")
df_combined = pd.concat([df_1, df_2, df_3])

In [19]:
df_combined

,Number,Title,Year,Runtime,Genre,Rating,Budget,Votes,Metascore
0,1.,The Hunger Games: The Ballad of Songbirds and ...,(2023),NaN,Action - Adventure - Drama,NaN,NaN,NaN,NaN
1,2.,The Flash,(2023),NaN,Action - Adventure - Fantasy,NaN,"$200,000,000",NaN,NaN
2,3.,Guardians of the Galaxy Vol. 3,(2023),150 min,Action - Adventure - Comedy,8.4,NaN,35997.0,65.0
3,4.,Ant-Man and the Wasp: Quantumania,(2023),124 min,Action - Adventure - Comedy,6.2,"$200,000,000",126234.0,48.0
4,5.,Transformers: Rise of the Beasts,(2023),NaN,Action - Adventure - Sci-Fi,NaN,"$200,000,000",NaN,NaN
...,...,...,...,...,...,...,...,...,...
1014,996,Teenage Mutant Ninja Turtles III,-1993,96 min,Action - Adventure - Comedy,4.8,"$21,000,000",34500.0,40.0
1015,997,It's a Boy Girl Thing,-2006,95 min,Comedy - Fantasy - Romance,6.2,"$15,500,000",41972.0,NaN
1016,998,Justice Society: World War II,-2021,84 min,Animation - Action - Adventure,6.4,NaN,8206.0,NaN
1017,999,Warlock,-1989,103 min,Action - Fantasy - Horror,6.2,"$7,000,000",17874.0,44.0


In [20]:
df_combined.info

<bound method DataFrame.info of         Number                                              Title    Year   
0           1.  The Hunger Games: The Ballad of Songbirds and ...  (2023)  \
1           2.                                          The Flash  (2023)   
2           3.                     Guardians of the Galaxy Vol. 3  (2023)   
3           4.                  Ant-Man and the Wasp: Quantumania  (2023)   
4           5.                   Transformers: Rise of the Beasts  (2023)   
...        ...                                                ...     ...   
1014       996                   Teenage Mutant Ninja Turtles III   -1993   
1015       997                              It's a Boy Girl Thing   -2006   
1016       998                      Justice Society: World War II   -2021   
1017       999                                            Warlock   -1989   
1018  1,000.00                                  A Christmas Carol   -2009   

      Runtime                             G

In [21]:
df_combined.describe()

,Rating,Votes,Metascore
count,7129.000000,7.129000e+03,3196.000000
mean,5.662379,7.611098e+04,55.689612
std,1.483821,1.642855e+05,17.421427
min,1.000000,5.000000e+00,6.000000
25%,4.700000,7.680000e+02,44.000000
50%,5.900000,4.958000e+03,56.000000
75%,6.700000,8.627900e+04,68.000000
max,9.900000,2.404440e+06,99.000000


In [22]:
df_combined.describe().T

,count,mean,std,min,25%,50%,75%,max
Rating,7129.0,5.662379,1.483821,1.0,4.7,5.9,6.7,9.9
Votes,7129.0,76110.975312,164285.459687,5.0,768.0,4958.0,86279.0,2404440.0
Metascore,3196.0,55.689612,17.421427,6.0,44.0,56.0,68.0,99.0


In [23]:
df_combined.isin([""]).sum()

Number       0
Title        0
Year         0
Runtime      0
Genre        0
Rating       0
Budget       0
Votes        0
Metascore    0
dtype: int64

In [24]:
df_combined.isnull().sum()

Number          0
Title           0
Year          307
Runtime       585
Genre           0
Rating        616
Budget       3768
Votes         616
Metascore    4549
dtype: int64

In [11]:
df

,Number,Title,Year,Runtime,Genre,Rating,Budget,Votes,"Metascore, Gross",Metascore,Gross
0,1.,The Wolf of Wall Street,(2013),180 min,Biography - Comedy - Crime,8.2,,1457185,NaN,75,
1,2.,Back to the Future,(1985),116 min,Adventure - Comedy - Sci-Fi,8.5,,1234604,NaN,87,
2,3.,Guardians of the Galaxy,(2014),121 min,Action - Adventure - Comedy,8.0,,1207207,NaN,76,
3,4.,The Truman Show,(1998),103 min,Comedy - Drama,8.2,,1113251,NaN,90,
4,5.,Up,(2009),96 min,Animation - Adventure - Comedy,8.3,,1066354,NaN,88,
5,6.,Deadpool,(2016),108 min,Action - Comedy,8.0,,1064310,NaN,65,
6,7.,Finding Nemo,(2003),100 min,Animation - Adventure - Comedy,8.2,,1063847,NaN,90,
7,8.,Toy Story,(1995),81 min,Animation - Adventure - Comedy,8.3,,1015167,NaN,96,
8,9.,"Monsters, Inc.",(2001),92 min,Animation - Adventure - Comedy,8.1,,929508,NaN,79,
9,10.,The Intouchables,(2011),112 min,Biography - Comedy - Drama,8.5,,879088,NaN,57,


In [8]:
from selenium import webdriver
#from selenium.webdriver.chrome.options import Options
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time
#from selenium.webdriver.chrome.service import Service
import requests
import re
import warnings

warnings.filterwarnings('ignore')

options = Options()
options.headless = True
driver = webdriver.Firefox(firefox_binary="C:/Program Files/LibreWolf/librewolf.exe", options=options, executable_path=GeckoDriverManager().install())


df = pd.DataFrame(columns=["Number", "Title", "Year", "Runtime", "Genre", "Rating", "Budget", "Votes", "Metascore", "Gross"])

url_template = "https://www.imdb.com/search/title/?genres=comedy&sort=num_votes,desc&start={}&explore=title_type,genres&title_type=movie&ref_=adv_explore_rhs&language=en"
start = 1
movies = []

# Only scrape data from the first 2 pages, run the loop twice
for start in range(1, 3001, 50):
    url = url_template.format(start)
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    items = soup.select('.lister-item-content')
    if not items:
        break

    for item in soup.select('.lister-item-content'):
        number = item.select_one('.lister-item-header span').text.strip('()')
        title = item.select_one('.lister-item-header a').text.strip()
        runtime_element = item.select_one('.runtime')
        runtime = runtime_element.text.strip() if runtime_element else ''
        rating_element = item.select_one('.ratings-imdb-rating strong')
        rating = rating_element.text.strip() if rating_element else ''
        year = item.select_one('.lister-item-content .text-muted').text.strip()

        genre_element = item.select_one('.genre')
        genre = " - ".join(genre_element.text.strip().split(',')) if genre_element else ''

        # Metascore bilgisini alınacaktır.
        metascore_element = item.select_one('.metascore')
        metascore = metascore_element.text.strip() if metascore_element else ''

        # Oyların sayısı alınacaktır.
        votes_element = item.select_one('span[name="nv"]')
        if votes_element:
            votes = votes_element['data-value']
        else:
            votes = ''
        
        gross_elements = item.select('span[name="nv"]')
        gross = ''
        for element in gross_elements:
            if re.search(r'\$\d+(\.\d+)?[M,B]$', element.text):
                gross = element['data-value']
                break

        # Budget değeri varsa alınacak eğer tahmini varsa o da alınacaktır.
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        budget_elements = soup.findAll('span', {'class': 'ipc-metadata-list-item__list-content-item'})
        budget = ''
        for element in budget_elements:
            text = element.text.strip()
            if text.startswith('$'):
                if any(x in text.lower() for x in ["gross worldwide", "gross us & canada", "opening weekend us & canada"]):
                    continue
                else:
                    if "Budget" in text.lower() or "estimated" in text.lower():
                        budget = text.split(' (')[0]
                        break
                    else:
                        budget = ''
            elif "Budget" in text.lower() or "estimated" in text.lower():
                    budget = text.split(' (')[0]
                    break
            else:
                budget = ''

        df = pd.concat([df, pd.DataFrame({"Number": number, "Title": title, "Year": year, "Runtime": runtime, "Genre": genre, "Rating": rating, "Budget": budget, "Votes": votes, "Metascore": metascore, "Gross": gross}, index=[0])], ignore_index=True)
        df.to_csv("imdb_scraped_data_comedy.csv", index=False)
driver.quit()

KeyboardInterrupt: 

In [3]:
df

,Number,Title,Year,Runtime,Genre,Rating,Budget,Votes,Metascore,Gross
0,1.,The Wolf of Wall Street,(2013),180 min,Biography - Comedy - Crime,8.2,,1457224,75,"116,900,694"
1,2.,Back to the Future,(1985),116 min,Adventure - Comedy - Sci-Fi,8.5,,1234630,87,"210,609,762"
2,3.,Guardians of the Galaxy,(2014),121 min,Action - Adventure - Comedy,8.0,,1207268,76,"333,176,600"
3,4.,The Truman Show,(1998),103 min,Comedy - Drama,8.2,,1113272,90,"125,618,201"
4,5.,Up,(2009),96 min,Animation - Adventure - Comedy,8.3,,1066381,88,"293,004,164"
...,...,...,...,...,...,...,...,...,...,...
110,111.,Knocked Up,(2007),129 min,Comedy - Romance,6.9,,375831,85,"148,734,225"
111,112.,Home Alone 2: Lost in New York,(1992),120 min,Adventure - Comedy - Crime,6.8,,374136,46,"173,585,516"
112,113.,Men in Black 3,(2012),106 min,Action - Adventure - Comedy,6.8,,372115,58,"179,020,854"
113,114.,Yes Man,(2008),104 min,Comedy - Romance,6.8,,370665,46,"97,690,976"


In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time
import re
import warnings
warnings.filterwarnings('ignore')

options = Options()
options.headless = True
driver = webdriver.Firefox(firefox_binary="C:/Program Files/LibreWolf/librewolf.exe", options=options, executable_path=GeckoDriverManager().install())

df = pd.DataFrame(columns=["Number", "Title", "Year", "Runtime", "Genre", "Rating", "Budget", "Gross US & Canada", "Votes", "Metascore"])
url_template = "https://www.imdb.com/search/title/?title_type=feature&genres=Drama&start={}&sort=boxoffice_gross_us,desc&explore=title_type, genres"
start = 1
movies = []

# sadece ilk 2 sayfanın verisini almak için döngüyü 2 kez çalıştırıyoruz
for start in range(1, 3001, 50):
    url = url_template.format(start)
    driver.get(url)
    time.sleep(6)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    items = soup.select('.lister-item-content')
    if not items:
        break


    for item in soup.select('.lister-item-content'):
        number = item.select_one('.lister-item-header span').text.strip('()')
        title = item.select_one('.lister-item-header a').text.strip()
        runtime_element = item.select_one('.runtime')
        runtime = runtime_element.text.strip() if runtime_element else ''
        rating_element = item.select_one('.ratings-imdb-rating strong')
        rating = rating_element.text.strip() if rating_element else ''
        year = item.select_one('.lister-item-content .text-muted').text.strip()

        genre_element = item.select_one('.genre')
        genre = " - ".join(genre_element.text.strip().split(',')) if genre_element else ''

        # Metascore bilgisini alınacaktır.
        metascore_element = item.select_one('.metascore')
        metascore = metascore_element.text.strip() if metascore_element else ''


        link = item.select_one('.lister-item-header a')['href']
        driver.get(f"https://www.imdb.com{link}business")
        time.sleep(1)
        
        gross_elements = item.select('span[name="nv"]')
        gross = ''
        for element in gross_elements:
            if re.search(r'\$\d+(\.\d+)?[M,B]$', element.text):
                gross = element['data-value']
                break

        soup = BeautifulSoup(driver.page_source, 'html.parser')
            
        # Oyların sayısı alınacaktır.
        votes_element = item.select_one('span[name="nv"]')
        if votes_element:
            votes = votes_element['data-value']
        else:
            votes = ''
        
        # Budget değeri varsa alınacak eğer tahmini varsa o da alınacaktır.
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        budget_elements = soup.findAll('span', {'class': 'ipc-metadata-list-item__list-content-item'})
        budget = ''
        for element in budget_elements:
            text = element.text.strip()
            if text.startswith('$'):
                if any(x in text.lower() for x in ["gross worldwide", "gross us & canada", "opening weekend us & canada"]):
                    continue
                else:
                    if "Budget" in text.lower() or "estimated" in text.lower():
                        budget = text.split(' (')[0]
                        break
                    else:
                        budget = ''
            elif "Budget" in text.lower() or "estimated" in text.lower():
                    budget = text.split(' (')[0]
                    break
            else:
                budget = ''

        df = pd.concat([df, pd.DataFrame({"Number": number, "Title": title, "Year": year, "Runtime": runtime, "Genre": genre, "Rating": rating, "Budget": budget, "Gross US & Canada": gross, "Votes": votes, "Metascore": metascore}, index=[0])], ignore_index=True)
        df.to_csv("imdb_scraped_data_drama.csv", index=False)
driver.quit()

[WDM] - Downloading: 19.2kB [00:00, 2.23MB/s]                                                                          
[WDM] - Downloading: 19.2kB [00:00, 6.57MB/s]                                                                          
[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 1.59M/1.59M [00:00<00:00, 13.4MB/s]


In [6]:
import pandas as pd
df = pd.concat(map(pd.read_csv, ['imdb_scraped_data_Horror.csv', 'imdb_scraped_data_Thriller.csv', 'imdb_scraped_data_action.csv', 'imdb_scraped_data_comedy.csv', 'imdb_scraped_data_crime.csv', 'imdb_scraped_data_drama.csv', 'imdb_scraped_data_fantasy.csv', 'imdb_scraped_data_scifi.csv']), ignore_index=True)
print(df)


      Number                            Title    Year  Runtime  \
0         1.                      The Shining  (1980)  146 min   
1         2.                            Alien  (1979)  117 min   
2         3.                      I Am Legend  (2007)  101 min   
3         4.                      World War Z  (2013)  116 min   
4         5.                           Psycho  (1960)  109 min   
...      ...                              ...     ...      ...   
13994   57.0                      World War Z  (2013)  116 min   
13995   58.0            2001: A Space Odyssey  (1968)  149 min   
13996   59.0  The Hunger Games: Catching Fire  (2013)  146 min   
13997   60.0           Spider-Man: Homecoming  (2017)  133 min   
13998   61.0                     Wonder Woman  (2017)  141 min   

                                Genre  Rating        Budget Gross US & Canada  \
0                     Drama -  Horror     8.4   $19,000,000        44,017,374   
1                    Horror -  Sci-Fi     8.5

In [7]:
df.duplicated(subset=['Title'])

0        False
1        False
2        False
3        False
4        False
         ...  
13994     True
13995    False
13996     True
13997     True
13998     True
Length: 13999, dtype: bool

In [8]:
print(len(df) - len(df.drop_duplicates()))

19


In [9]:
df

,Number,Title,Year,Runtime,Genre,Rating,Budget,Gross US & Canada,Votes,Metascore
0,1.,The Shining,(1980),146 min,Drama - Horror,8.4,"$19,000,000","44,017,374",1046194.0,66.0
1,2.,Alien,(1979),117 min,Horror - Sci-Fi,8.5,"$11,000,000","78,900,000",901097.0,89.0
2,3.,I Am Legend,(2007),101 min,Action - Drama - Horror,7.2,"$150,000,000","256,386,216",778116.0,65.0
3,4.,World War Z,(2013),116 min,Action - Adventure - Horror,7.0,"$190,000,000","202,359,711",685918.0,63.0
4,5.,Psycho,(1960),109 min,Horror - Mystery - Thriller,8.5,"$806,947","32,000,000",685911.0,97.0
...,...,...,...,...,...,...,...,...,...,...
13994,57.0,World War Z,(2013),116 min,Action - Adventure - Horror,7.0,"$190,000,000","202,359,711",685861.0,63.0
13995,58.0,2001: A Space Odyssey,(1968),149 min,Adventure - Sci-Fi,8.3,"$12,000,000","56,954,992",683552.0,84.0
13996,59.0,The Hunger Games: Catching Fire,(2013),146 min,Action - Adventure - Sci-Fi,7.5,"$130,000,000","424,668,047",682086.0,76.0
13997,60.0,Spider-Man: Homecoming,(2017),133 min,Action - Adventure - Sci-Fi,7.4,"$175,000,000","334,201,140",677261.0,73.0


In [16]:
print(len(df) - len(df[df['Gross US & Canada'].isna() | df['Budget'].isna() | df['Metascore'].isna() ]))

8712
